In [10]:
import cv2
import numpy as np


# declaring min. width and height
min_width_react = 80   #min. width reactangle
min_hieght_react = 80   #min. height reactangle

# web camera 
cap = cv2.VideoCapture("C:/Users/admin/Desktop/cars.mp4")


count_line_postion = 550

# Initialize substructor
# algo = cv2.bgsegm.createBackgroundSubtractorMOG()
algo = cv2.createBackgroundSubtractorKNN()
# algo = cv2.createBackgroundSubtractorMOG2()

# detector = cv2.CascadeClassifier("C:/Users/admin/Desktop/Coaching/Data Science/cars.xml")

def center_handle(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x + x1
    cy = y + y1
    return cx, cy

detect = []
offset = 6      # allowable error between pixel
counter = 0


while True:
    ret, frame = cap.read()
    if(type(frame)==type(None)):
        break
    grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3,3), 5)
    # applying on each frame 
    img_sub = algo.apply(blur)                # appling algorithm
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))            
    '''the function dilates the source image using the specified structuring element
    that determines the shape of a pixel neighborhood over which the maximum is taken:'''

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    '''This function constructs and returns the structuring element 
    that can be further passed to #erode, #dilate'''

    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    '''morphologyEx: This function is used to give the shape of your object 
    like it white the object i.e. car'''
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)

    counterSahpe,h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    '''findContours: is used to fine the images'''

    '''draw the line on highway'''
    cv2.line(frame, (25,count_line_postion), (1200,count_line_postion), (255,127,0), 3)

    '''draw reactangle onto the frame'''

    # cars = detector.detectMultiScale(frame,5.5,1) # trus hold value = 5.5
    # for (x,y,w,h) in cars:
    #     cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    
    for (i,c) in enumerate(counterSahpe):
        (x,y,w,h) = cv2.boundingRect(c)
        validate_counter = (w>= min_width_react) and (h>= min_hieght_react)
        if not validate_counter:
            continue
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)   
        #             img,  pt1,  pt2     , color,   thickness
        cv2.putText(frame, "Vehicle"+str(counter), (x, y-20), cv2.FONT_HERSHEY_TRIPLEX, 1, (255,244,0), 2)


#detect the cars 
        center = center_handle(x,y,w,h)
        detect.append(center)
        cv2.circle(frame, center, 4, (0,255,255), -1)

# detectd no. cars 
        for (x,y) in detect:
            if y<(count_line_postion + offset) and y>(count_line_postion - offset):
                counter += 1
                cv2.line(frame, (25,count_line_postion), (1200,count_line_postion), (0,127,255), 3)
                detect.remove((x,y))
#                 print("Vehicle Counter: "+str(counter))

# detectd no. cars and show on the screen
    cv2.putText(frame, "Vehicle Counter: "+str(counter), (450,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5)


    # cv2.imshow('Detecter', dilatada)
    cv2.imshow('Car Detection', frame)

    if cv2.waitKey(30) == ord('q'):
        break
        
    
print("Total Vehicle passed: "+str(counter))
cv2.destroyAllWindows()
cap.release()



Total Vehicle passed: 6
